In [1]:
import os.path

import cirq
import quantumcircuitbenchmarks.cirq as qcb
class generators(int):
    def bv(n, **kwargs):
        return qcb.generate_bv([1]*(n-1), **kwargs)
    def cnx_halfborrowed(n, **kwargs):
        '''Only odd number of qubits supported.'''
        rn = (n + 3) // 2
        assert n == rn+rn-3, 'Unsupported number of qubits'
        return qcb.generate_cnu_halfborrowed(rn, **kwargs)
    def cnx_dirty(n, m=None, **kwargs):
        '''Even n will be longer circuits (if m is unspecified)'''
        if m is None:
            rn = (n + 4) // 2
            m = rn - 3 if n % 2 == 1 else rn - 4
            assert rn + m == n, f'{rn}, {n}, {m}'
        else:
            rn = n
        return qcb.generate_dirty_multicontrol(rn, m, **kwargs)
    cnx_inplace = qcb.generate_cnx_linear
    cnx_logancilla = qcb.generate_cnx_log_depth
    #cnx_any = qcb.generate_cnx_n_m
    def grovers(n, m=None, num_rounds=None, **kwargs):
        '''Even n will be longer circuits (if m is unspecified)'''
        if m is None:
            rn = (n + 4) // 2
            m = rn - 3 if n % 2 == 1 else rn - 4
            assert rn + m == n, f'{rn}, {n}, {m}'
        else:
            rn = n
        return qcb.generate_grover_integer_search_circuit(
                    rn, m, 2**rn-1, num_rounds=num_rounds, **kwargs)
    def incrementer_borrowedbit(r, **kwargs):
        return qcb.generate_incrementer_borrowedbit(r-1, **kwargs)
    def qaoa_random(n, prob=0.5, p=1, seed=0, **kwargs):
        return qcb.generate_random_QAOA(n, prob, p, seed=seed, **kwargs)
    def qaoa_random1(n, prob=0.05, p=1, seed=1, **kwargs):
        return qcb.generate_random_QAOA(n, prob, p, seed=seed, **kwargs)
    def qaoa_random2(n, prob=0.1, p=1, seed=2, **kwargs):
        return qcb.generate_random_QAOA(n, prob, p, seed=seed, **kwargs)
    def qaoa_random3(n, prob=0.15, p=1, seed=3, **kwargs):
        return qcb.generate_random_QAOA(n, prob, p, seed=seed, **kwargs)
    def qaoa_complete(n, prob=1, p=1, seed=0, **kwargs):
        return qcb.generate_random_QAOA(n, prob, p, seed=seed, **kwargs)
    #qaoa = qcb.generate_QAOA_circuit
    cuccaro_adder = qcb.generate_cuccaro_adder
    qft_adder = qcb.generate_qft_adder
    takahashi_adder = qcb.generate_takahashi_adder

In [2]:
def save_circuit_interactions(c, fname, assert_width=None, allow_toffoli=False):
    qubits = sorted(c.all_qubits())
    if assert_width is not None:
        assert len(qubits) == assert_width, (
            f'Not the expected circuit width: {len(qubits)} != {assert_width}')
    qubit_map = {q: i for i, q in enumerate(qubits)}
    interactions = []
    used_set = set(range(len(qubits)))
    depth = 0
    for op in c.all_operations():
        if len(op.qubits) < 2: continue
        if isinstance(op.gate, (cirq.MeasurementGate, cirq.ResetChannel)):
            continue
        if len(op.qubits) > 2+bool(allow_toffoli):
            raise ValueError(f'Operation on more than two qubits: {op}')
        qubit_ids = [qubit_map[q] for q in op.qubits]
        if any(qi in used_set for qi in qubit_ids):
            used_set.clear()
            depth += 1
        used_set.update(qubit_ids)
        interactions.append(f'{" ".join(map(str, qubit_ids))}\n')
    with open(fname, 'w') as f:
        f.write('===\n')
        f.write(f'n={len(qubits)}\n')
        f.write(f'depth={depth}\n')
        f.write(f'gates={len(interactions)}\n')
        f.write('---\n')
        for line in interactions:
            f.write(line)
        f.write('===\n')
    return len(qubits), depth, len(interactions)
#save_circuit_interactions(generators.bv(5), 'benchmark_files/bv.cint', 5)

In [3]:
def save(dirname, name, n):
    func = getattr(generators, name)
    print(f'Generating "{name}" for {n} qubits...')
    c2 = func(n)
    fname = os.path.join(dirname, f'{name}-{n}.cint')
    ret = save_circuit_interactions(c2, fname, assert_width=n)
    print(f'  Saved "{fname}"')
    c3 = func(n, to_toffoli=True)
    fname = os.path.join(dirname, f'{name}-{n}-compile3.cint')
    ret3 = save_circuit_interactions(c3, fname, assert_width=n, allow_toffoli=True)
    print(f'  Saved "{fname}"')
    cnot_count = 0
    for op in c2.all_operations():
        assert len(op.qubits) <= 2
        cnot_count += len(op.qubits) == 2
    toff_count = 0
    for op in c3.all_operations():
        assert len(op.qubits) <= 3
        toff_count += len(op.qubits) == 3
    qubit_count = len(c2.all_qubits())
    print(f'  {qubit_count:-3} & {toff_count:-3} & {cnot_count:-3}')
    return ret

In [4]:
for k in generators.__dict__.keys():
    if k.startswith('_'): continue
    print(k)

bv
cnx_halfborrowed
cnx_dirty
cnx_inplace
cnx_logancilla
grovers
incrementer_borrowedbit
qaoa_random
qaoa_random1
qaoa_random2
qaoa_random3
qaoa_complete
cuccaro_adder
qft_adder
takahashi_adder


In [5]:
max_n = 20
save('benchmark_files', 'bv', max_n)

Generating "bv" for 20 qubits...
  Saved "benchmark_files/bv-20.cint"
  Saved "benchmark_files/bv-20-compile3.cint"
   20 &   0 &  19


(20, 19, 19)

In [6]:
save('benchmark_files', 'cnx_halfborrowed', max_n-1)

Generating "cnx_halfborrowed" for 19 qubits...
  Saved "benchmark_files/cnx_halfborrowed-19.cint"
  Saved "benchmark_files/cnx_halfborrowed-19-compile3.cint"
   19 &  32 & 256


(19, 238, 256)

In [7]:
save('benchmark_files', 'cnx_dirty', 11)

Generating "cnx_dirty" for 11 qubits...
  Saved "benchmark_files/cnx_dirty-11.cint"
  Saved "benchmark_files/cnx_dirty-11-compile3.cint"
   11 &  16 & 128


(11, 118, 128)

In [8]:
save('benchmark_files', 'cnx_logancilla', max_n-1)

Generating "cnx_logancilla" for 19 qubits...
  Saved "benchmark_files/cnx_logancilla-19.cint"
  Saved "benchmark_files/cnx_logancilla-19-compile3.cint"
   19 &  17 & 136


(19, 52, 136)

In [9]:
r = None
if max_n-1 <= 22:
    r = save('benchmark_files', 'cnx_inplace', 4)
r

Generating "cnx_inplace" for 4 qubits...
  Saved "benchmark_files/cnx_inplace-4.cint"
  Saved "benchmark_files/cnx_inplace-4-compile3.cint"
    4 &  54 & 490


(4, 485, 490)

In [10]:
save('benchmark_files', 'qaoa_random1', max_n)

Generating "qaoa_random1" for 20 qubits...
  Saved "benchmark_files/qaoa_random1-20.cint"
  Saved "benchmark_files/qaoa_random1-20-compile3.cint"
   20 &   0 &  20


(20, 8, 20)

In [11]:
save('benchmark_files', 'qaoa_random2', max_n)

Generating "qaoa_random2" for 20 qubits...
  Saved "benchmark_files/qaoa_random2-20.cint"
  Saved "benchmark_files/qaoa_random2-20-compile3.cint"
   20 &   0 &  30


(20, 10, 30)

In [12]:
save('benchmark_files', 'qaoa_random3', max_n)

Generating "qaoa_random3" for 20 qubits...
  Saved "benchmark_files/qaoa_random3-20.cint"
  Saved "benchmark_files/qaoa_random3-20-compile3.cint"
   20 &   0 &  58


(20, 18, 58)

In [13]:
save('benchmark_files', 'qaoa_complete', 10)

Generating "qaoa_complete" for 10 qubits...
  Saved "benchmark_files/qaoa_complete-10.cint"
  Saved "benchmark_files/qaoa_complete-10-compile3.cint"
   10 &   0 &  90


(10, 34, 90)

In [14]:
save('benchmark_files', 'cuccaro_adder', max_n)

Generating "cuccaro_adder" for 20 qubits...
  Saved "benchmark_files/cuccaro_adder-20.cint"
  Saved "benchmark_files/cuccaro_adder-20-compile3.cint"
   20 &  18 & 190


(20, 165, 190)

In [15]:
save('benchmark_files', 'qft_adder', 16)

Generating "qft_adder" for 16 qubits...
  Saved "benchmark_files/qft_adder-16.cint"
  Saved "benchmark_files/qft_adder-16-compile3.cint"
   16 &   0 &  92


(16, 29, 92)

In [16]:
save('benchmark_files', 'takahashi_adder', max_n)

Generating "takahashi_adder" for 20 qubits...
  Saved "benchmark_files/takahashi_adder-20.cint"
  Saved "benchmark_files/takahashi_adder-20-compile3.cint"
   20 &  18 & 188


(20, 171, 188)

In [17]:
save('benchmark_files', 'incrementer_borrowedbit', 5)

Generating "incrementer_borrowedbit" for 5 qubits...
  Saved "benchmark_files/incrementer_borrowedbit-5.cint"
  Saved "benchmark_files/incrementer_borrowedbit-5-compile3.cint"
    5 &  50 & 448


(5, 442, 448)

In [18]:
save('benchmark_files', 'grovers', 9)

Generating "grovers" for 9 qubits...
  Saved "benchmark_files/grovers-9.cint"
  Saved "benchmark_files/grovers-9-compile3.cint"
    9 &  84 & 672


(9, 455, 672)

In [19]:
generators(5).bv()

0: ───H───────@───H───────────────
              │
1: ───H───────┼───@───H───────────
              │   │
2: ───H───────┼───┼───@───H───────
              │   │   │
3: ───H───────┼───┼───┼───@───H───
              │   │   │   │
4: ───X───H───X───X───X───X───H───

In [20]:
generators(5).cnx_halfborrowed()  # Must be odd

┌──┐                                    ┌──┐
0: ───T───X─────────────────────────────────────────────────────@─────T^-1───X───@───X──────@───X───────@─────T───X────────────────────────────────────────────────────@───T^-1───X───@───X──────@───X──────@───────
          │                                                     │            │   │   │      │   │       │         │                                                    │          │   │   │      │   │      │
1: ───T───@─────────────────────────────────────────────────────┼────────────@───┼───@──────┼───@───────┼T────────@────────────────────────────────────────────────────┼──────────@───┼───@──────┼───@──────┼───────
                                                                │                │          │           │                                                              │              │          │          │
2: ───T───────────X───T──────────X───T^-1───X───T^-1───X───T────┼────────────────┼──────────┼───────────┼─────────────────X───T──────────X───T^-1───X───T^-1───X───────┼──────────────┼──────────┼──────────┼───────
                  │              │          │          │        │                │          │           │                 │              │          │          │       │              │          │          │
3: ───H───T───X───@───T^-1───X───@───X──────@───X──────@───H────┼H────T──────────┼──────────┼───────────┼─────────────X───@───T^-1───X───@───X──────@───X──────@───H───┼──────────────┼──────────┼──────────┼───────
              │              │       │          │               │                │          │           │             │              │       │          │              │              │          │          │
4: ───T───────@──────────────@───────@──────────@──────H───T────X─────T──────────X───T^-1───X───T^-1────X─────H───T───@──────────────@───────@──────────@──────H───T───X───T──────────X───T^-1───X───T^-1───X───H───
                                                               └──┘                                    └──┘

In [21]:
generators(5).cnx_dirty(to_toffoli=False)  # Better when odd

┌──┐                                    ┌──┐
0: ───T───X─────────────────────────────────────────────────────@─────T^-1───X───@───X──────@───X───────@─────T───X────────────────────────────────────────────────────@───T^-1───X───@───X──────@───X──────@───────
          │                                                     │            │   │   │      │   │       │         │                                                    │          │   │   │      │   │      │
1: ───T───@─────────────────────────────────────────────────────┼────────────@───┼───@──────┼───@───────┼T────────@────────────────────────────────────────────────────┼──────────@───┼───@──────┼───@──────┼───────
                                                                │                │          │           │                                                              │              │          │          │
2: ───T───────────X───T──────────X───T^-1───X───T^-1───X───T────┼────────────────┼──────────┼───────────┼─────────────────X───T──────────X───T^-1───X───T^-1───X───────┼──────────────┼──────────┼──────────┼───────
                  │              │          │          │        │                │          │           │                 │              │          │          │       │              │          │          │
3: ───H───T───X───@───T^-1───X───@───X──────@───X──────@───H────┼H────T──────────┼──────────┼───────────┼─────────────X───@───T^-1───X───@───X──────@───X──────@───H───┼──────────────┼──────────┼──────────┼───────
              │              │       │          │               │                │          │           │             │              │       │          │              │              │          │          │
4: ───T───────@──────────────@───────@──────────@──────H───T────X─────T──────────X───T^-1───X───T^-1────X─────H───T───@──────────────@───────@──────────@──────H───T───X───T──────────X───T^-1───X───T^-1───X───H───
                                                               └──┘                                    └──┘

In [22]:
generators(5).cnx_inplace()

┌──┐                                                                                                        ┌──┐                                                                                                             ┌──┐                                                                                                        ┌──┐                                                                            ┌──┐                                                                                                                                                                              ┌──┐                                                    ┌──┐                                                                                                                                                                              ┌──┐                                                                                                         ┌──┐                                                                                                     ┌──┐           ┌──┐   ┌──┐                                                    ┌──┐                                                                                                                                                                              ┌──┐                                                    ┌──┐                                                                                                                                                                              ┌──┐                                                                                                         ┌──┐                                                                                                     ┌──┐                                                                    ┌──┐                                                                                                                                                                              ┌──┐                                                    ┌──┐                                                                                                                                                                              ┌──┐                                                        ┌──┐           ┌──┐   ┌──┐                                                    ┌──┐                                                                                                                                                                              ┌──┐                                                    ┌──┐                                                                                                                                                                              ┌──┐                                                        ┌──┐                                                                                                                         ┌──┐                                                            ┌──┐                                                        ┌──┐                                                                                                             ┌──┐                               ┌──┐                                    ┌──┐                                                        ┌──┐                     ┌──┐   ┌──┐                                                    ┌──┐                                                                                                                                                                              ┌──┐                                                    ┌──┐                                                                                                                                                                              ┌──┐                                                                                                         ┌──┐                                                         

In [23]:
generators(5).cnx_logancilla()

┌─────┐
0: ───T───X───@───T^-1───X───@───X──────@───X───────@────────T^-1───X───────────────────────────────────────────────────────@───T──────X───@───X───@───X───@───────
          │   │          │   │   │      │   │       │               │                                                       │          │   │   │   │   │   │
1: ───T───@───┼──────────@───┼───@──────┼───@───────┼T^-1───────────@───────────────────────────────────────────────────────┼──────────@───┼───@───┼───@───┼───────
              │              │          │           │                                                                       │              │       │       │
2: ───T───────┼──────────────┼──────────┼───────────┼───────────────────────X───T──────────X───T^-1───X───T^-1───X──────────┼──────────────┼───────┼───────┼───────
              │              │          │           │                       │              │          │          │          │              │       │       │
3: ───H───T───┼──────────────┼──────────┼───────────┼───────────────────X───@───T^-1───X───@───X──────@───X──────@───H──────┼──────────────┼───────┼───────┼───────
              │              │          │           │                   │              │       │          │                 │              │       │       │
4: ───H───T───X───T──────────X───T^-1───X───T^-1────X────────H──────T───@──────────────@───────@──────────@──────H───T^-1───X───T^-1───────X───T───X───T───X───H───
                                                   └─────┘

In [24]:
#generators.cnx_any(4, 1)

In [25]:
generators(5).grovers()

┌──┐                                    ┌──┐                                                            ┌──┐              ┌──┐          ┌──┐          ┌──┐                                                        ┌──┐                                                            ┌──┐              ┌──┐          ┌──┐          ┌──┐                                                        ┌──┐                                                            ┌──┐              ┌──┐          ┌──┐          ┌──┐                                                        ┌──┐                                                            ┌──┐              ┌──┐          ┌──┐          ┌──┐                                                        ┌──┐                                                            ┌──┐              ┌──┐          ┌──┐          ┌──┐                                                        ┌──┐                                                            ┌──┐                                    ┌──┐
0: ───H───T───X────@─────T^-1───X───@───X──────@───X───────@─────T───X─────────────────────────────────────────────────────@─────T^-1───X────@─────X───────@─────X───────@─────H───X───T───X───@───T^-1───X───@───X──────@───X───────@─────T───X─────────────────────────────────────────────────────@─────T^-1───X────@─────X───────@─────X───────@─────X───H───T───X───@───T^-1───X───@───X──────@───X───────@─────T───X─────────────────────────────────────────────────────@─────T^-1───X────@─────X───────@─────X───────@─────H───X───T───X───@───T^-1───X───@───X──────@───X───────@─────T───X─────────────────────────────────────────────────────@─────T^-1───X────@─────X───────@─────X───────@─────X───H───T───X───@───T^-1───X───@───X──────@───X───────@─────T───X─────────────────────────────────────────────────────@─────T^-1───X────@─────X───────@─────X───────@─────H───X───T───X───@───T^-1───X───@───X──────@───X───────@─────T───X─────────────────────────────────────────────────────@─────T^-1───X───@───X──────@───X───────@─────X───H───
              │    │            │   │   │      │   │       │         │                                                     │            │    │     │       │     │       │                 │   │          │   │   │      │   │       │         │                                                     │            │    │     │       │     │       │                 │   │          │   │   │      │   │       │         │                                                     │            │    │     │       │     │       │                 │   │          │   │   │      │   │       │         │                                                     │            │    │     │       │     │       │                 │   │          │   │   │      │   │       │         │                                                     │            │    │     │       │     │       │                 │   │          │   │   │      │   │       │         │                                                     │            │   │   │      │   │       │
1: ───H───T───@────┼────────────@───┼───@──────┼───@───────┼T────────@─────────────────────────────────────────────────────┼────────────@────┼─────@───────┼─────@───────┼H────X───T───────@───┼──────────@───┼───@──────┼───@───────┼T────────@─────────────────────────────────────────────────────┼────────────@────┼─────@───────┼─────@───────┼X────H───T───────@───┼──────────@───┼───@──────┼───@───────┼T────────@─────────────────────────────────────────────────────┼────────────@────┼─────@───────┼─────@───────┼H────X───T───────@───┼──────────@───┼───@──────┼───@───────┼T────────@─────────────────────────────────────────────────────┼────────────@────┼─────@───────┼─────@───────┼X────H───T───────@───┼──────────@───┼───@──────┼───@───────┼T────────@─────────────────────────────────────────────────────┼────────────@────┼─────@───────┼─────@───────┼H────X───T───────@───┼──────────@───┼───@──────┼───@───────┼T────────@──────────────────────────────────────

In [26]:
generators(5).incrementer_borrowedbit()

┌──┐   ┌──┐                                                    ┌──┐                                                                                                                                                                              ┌──┐                                                    ┌──┐                                                                                                                                                                              ┌──┐                                                                                                         ┌──┐                                                                                                     ┌──┐           ┌──┐   ┌──┐                                                    ┌──┐                                                                                                                                                                              ┌──┐                                                    ┌──┐                                                                                                                                                                              ┌──┐                                                                                                         ┌──┐                                                                                                     ┌──┐                                                                    ┌──┐                                                                                                                                                                              ┌──┐                                                    ┌──┐                                                                                                                                                                              ┌──┐                                                        ┌──┐           ┌──┐   ┌──┐                                                    ┌──┐                                                                                                                                                                              ┌──┐                                                    ┌──┐                                                                                                                                                                              ┌──┐                                                        ┌──┐                                                                                                                         ┌──┐                                                            ┌──┐                                                        ┌──┐                                                                                                             ┌──┐                                                            ┌──┐                                                        ┌──┐
0: ────@──────@─────H───T───X───@───T^-1───X───@───X──────@───X───────@─────H───T───@──────────────@───────@──────────@──────T───────────────────@──────────────@───────@──────────@──────H───T───────X───@───T^-1───X───@───X──────@───X──────@───H─────@────H───T───X───@───T^-1───X───@───X──────@───X───────@─────H───T───@──────────────@───────@──────────@──────T───────────────────@──────────────@───────@──────────@──────H───T───────X───@───T^-1───X───@───X──────@───X──────@───H────@─────T───X────────────────────────────────────────────────────────────@───T^-1───X───@───X──────@───X───────@─────T───X────────────────────────────────────────────────────────@───T^-1───X───@───X──────@───X───────@───────────────@──────@────H───T───X───@───T^-1───X───@───X──────@───X───────@─────H───T───@──────────────@───────@──────────@──────T───────────────────@──────────────@───────@──────────@──────H───T───────X───@───T^-1───X───@───X──────@───X──────@───H─────@────H───T───X───@───T^-1───X───@───X──────@───X───────@─────H───T───

In [27]:
generators(5).qaoa_random1()
generators(5).qaoa_random2()
generators(5).qaoa_random3()

┌──┐           ┌──┐
0: ────@─────────@────@─────────@───X───────────────
       │         │    │         │
1: ────┼X────────┼────┼─────────┼───────────────────
       │         │    │         │
2: ────X─────Z───X────┼X────────┼───────────────────
                      │         │
3: ───────────────────┼─────────┼───@───────@───X───
                      │         │   │       │
4: ───────────────────X─────Z───X───X───Z───X───X───
      └──┘           └──┘

In [28]:
generators(5).qaoa_complete()

┌──┐       ┌──┐   ┌──┐       ┌──┐   ┌──┐       ┌──┐
0: ───@───────@───@───────@────@──────────@──────@──────────@──────X────────────────────────────────────────────
      │       │   │       │    │          │      │          │
1: ───X───Z───X───┼───────┼────┼@─────────┼@─────┼@─────────┼@─────@──────────@─────X───────────────────────────
                  │       │    ││         ││     ││         ││     │          │
2: ───────────────X───Z───X────┼X────Z────┼X─────┼┼─────────┼┼─────┼@─────────┼@────@───────@───X───────────────
                               │          │      ││         ││     ││         ││    │       │
3: ────────────────────────────X─────Z────X──────┼X────Z────┼X─────┼X────Z────┼X────┼───────┼───@───────@───X───
                                                 │          │      │          │     │       │   │       │
4: ──────────────────────────────────────────────X─────Z────X──────X─────Z────X─────X───Z───X───X───Z───X───X───
                              └──┘       └──┘   └──┘       └──┘   └──┘       └──┘

In [29]:
generators(6).cuccaro_adder()  # Must be even

┌──┐                                                                ┌──┐                                                            ┌──┐
0: ──────────X───T───────@──────────────@───────@──────────@────────────────@────T────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────@──────────────@───────@──────────@──────────────X───────
             │           │              │       │          │                │                                                                                                                                         │              │       │          │              │
1: ────@─────@───H───T───X───@───T^-1───X───@───X──────@───X──────@───H────X┼────T───────@──────────────@───────@──────────@────────────────@────T───────@──────────────@───────@──────────@──────────────X───H───T───X───@───T^-1───X───@───X──────@───X──────@───H───@───@───
       │                     │              │          │          │        ││            │              │       │          │                │            │              │       │          │              │               │              │          │          │           │
2: ────┼@────────────────────┼──────────────┼──────────┼──────────┼────────@┼────H───T───X───@───T^-1───X───@───X──────@───X──────@───H────@┼────H───T───X───@───T^-1───X───@───X──────@───X──────@───H───@───@───────────┼──────────────┼──────────┼──────────┼───────────┼───
       ││                    │              │          │          │         │                │              │          │          │        ││                │              │          │          │           │           │              │          │          │           │
3: ────X┼────T───────────────X───T──────────X───T^-1───X───T^-1───X───X─────X────T───────────┼──────────────┼──────────┼──────────┼────────┼┼────────────────┼──────────────┼──────────┼──────────┼───────────┼───────────X───T──────────X───T^-1───X───T^-1───X───X───────X───
        │                                                                                    │              │          │          │        ││                │              │          │          │           │
4: ─────X────T───────────────────────────────────────────────────────────────────────────────X───T──────────X───T^-1───X───T^-1───X───X────┼X────T───────────X───T──────────X───T^-1───X───T^-1───X───X───────X────────────────────────────────────────────────────────────────
                                                                                                                                           │
5: ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      └──┘                                                                └──┘                                                            └──┘

In [30]:
generators(6).qft_adder()  # Must be even

┌───────┐                ┌──────┐                     ┌────────┐
0: ────────────────@──────────@────────H────@─────────H───────@───────────@───────────────────────
                   │          │             │                 │           │
1: ───────@────────┼─────H────@^0.25───@────┼─────────@───────@^-0.25────H┼──────────@────────────
          │        │                   │    │         │                   │          │
2: ───H───@^0.5────@^0.25─────@────────┼────┼@────────┼───────@───────────@^-0.25────@^-0.5───H───
                              │        │    ││        │       │
3: ───────────────────────────┼────────┼────@┼────────@^0.5───@^0.25──────────────────────────────
                              │        │     │
4: ───────────────────────────┼────────@─────@^0.5────────────────────────────────────────────────
                              │
5: ───────────────────────────@───────────────────────────────────────────────────────────────────
                  └───────┘                └──────┘                     └────────┘

In [31]:
generators(6).takahashi_adder()  # Must be even

┌───┐                                                                                                                                                                              ┌──┐                                                        ┌──┐       ┌──┐
0: ────T──────────────────@──────────────@───────@──────────@──────T────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────@──────────────@───────@──────────@────────────@───────────────
                          │              │       │          │                                                                                                                                           │              │       │          │            │
1: ────@──────@───H───T───X───@───T^-1───X───@───X──────@───X──────@───H───T───@──────────────@───────@──────────@──────T───────────────────@──────────────@───────@──────────@────────────@────H───T───X───@───T^-1───X───@───X──────@───X──────@────H┼────@────@─────
       │      │               │              │          │          │           │              │       │          │                          │              │       │          │            │                │              │          │          │     │    │    │
2: ────┼@─────X───H───T───────┼──────────────┼──────────┼──────────┼───────────X───@───T^-1───X───@───X──────@───X──────@───H───@───H───T───X───@───T^-1───X───@───X──────@───X──────@────H┼────────────────┼──────────────┼──────────┼──────────┼─────┼────X────┼@────
       ││                     │              │          │          │               │              │          │          │       │               │              │          │          │     │                │              │          │          │     │         ││
3: ────┼┼T────────────────────X───T──────────X───T^-1───X───T^-1───X───T───────────┼──────────────┼──────────┼──────────┼───────┼───────────────┼──────────────┼──────────┼──────────┼─────┼────────────────X───T──────────X───T^-1───X───T^-1───X─────X─────────┼┼────
       ││                                                                          │              │          │          │       │               │              │          │          │     │                                                                     ││
4: ────X┼─────T────────────────────────────────────────────────────────────────────X───T──────────X───T^-1───X───T^-1───X───T───┼───────────────X───T──────────X───T^-1───X───T^-1───X─────X─────────────────────────────────────────────────────────────────────X┼────
        │                                                                                                                       │                                                                                                                                 │
5: ─────X───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X────
      └───┘                                                                                                                                                                              └──┘                                                        └──┘       └──┘

In [32]:
import cirq

In [33]:
cirq.Circuit(cirq.decompose(cirq.CCZ(*cirq.LineQubit.range(3))))

0: ───T────────────@────────────────────────────────@─────────────────────────────@────────────────────────────@───────────────────────────
                   │                                │                             │                            │
1: ───T───Y^-0.5───@───Y^0.5───@───T^-1────Y^-0.5───@────────Y^0.5───@───Y^-0.5───@──────Y^0.5────@───Y^-0.5───@──────Y^0.5────@───────────
                               │                                     │                            │                            │
2: ───T───Y^-0.5───────────────@───Y^0.5───T────────Y^-0.5───────────@───Y^0.5────T^-1───Y^-0.5───@───Y^0.5────T^-1───Y^-0.5───@───Y^0.5───